In [17]:
import pandas as pd
import plotly.express as px
import re
import datetime
import numpy as np

from Project.Database import Db

# Example data for the TPM

In [18]:
# pd.read_csv("/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/Data/ISID.csv")

In [19]:
# pd.read_json(
#     path_or_buf="/Users/christiannielsen/Documents/GitHub/Datavidenskab-P6/Project/TPM/output/Experiment_minsup0.1_minconf_0.1/level1.json",
#     orient='name_node')

In [20]:
year1_minutes, meta = Db.load_data(hourly=False, meta=True, year=1)

In [21]:
appliance_condition = (lambda self: (
        (self["Parameter"] == "Status_OnOff") &
        (self["Subsystem"] == "Loads") &
        (~self.index.str.contains("Child")) &
        (~self.index.str.contains("Prnt")) &
        (~self.index.str.contains("StatusLatentload"))
))

appliances_statusOnOff = meta.loc[appliance_condition].index.tolist()

In [22]:
#Replaces values to be binary 0 or 1.
for i in appliances_statusOnOff:
    year1_minutes.loc[year1_minutes[i] < 0, i] = 0
    year1_minutes.loc[year1_minutes[i] > 0, i] = 1

In [23]:
minute_appliances_status = year1_minutes[["Timestamp"] + appliances_statusOnOff].copy()

In [24]:
for col in meta["Measurement_Location"].unique():
    appliance_condition = (lambda self: (
            (self["Parameter"] == "Status_OnOff") &
            (self.index.str.contains("Light")) &
            (self["Measurement_Location"] == col)
    ))
    light_cols = meta.loc[appliance_condition].index
    if len(light_cols) > 1:
        minute_appliances_status[col + " Lights"] = minute_appliances_status[light_cols].max(1)
        minute_appliances_status.drop(light_cols, axis= 1, inplace=True)
        for attribute in light_cols:
            appliances_statusOnOff.remove(attribute)
        appliances_statusOnOff.append(col + " Lights")

In [25]:
#Forces Timestamp to the type of datetime, to extract the hour of Timestamp.
minute_appliances_status["Timestamp"] = pd.to_datetime(minute_appliances_status["Timestamp"],
                                                       format="%Y-%m-%d %H:%M:%S%z", utc=True) - pd.to_timedelta(
    unit="h", arg=(
        minute_appliances_status["Timestamp"].astype("str").str.split(pat="-", expand=True)[lambda self: self.columns[-1]].str[
            1]).astype(
        int))

#Extracts hour of Timestamp.
minute_appliances_status["DayOfYear"] = minute_appliances_status.Timestamp.dt.dayofyear

In [26]:
# minute_appliances_status['Timestamp'] = (pd.to_datetime(minute_appliances_status["Timestamp"],
#                                                        format="%Y-%m-%d %H:%M:%S%z", utc=True).astype("int") / 1e9).astype("int")

In [27]:
minute_appliances_status['Timestamp'] = (pd.DatetimeIndex ( minute_appliances_status["Timestamp"] ).astype ( np.int64 )/1e9).astype(int)


In [28]:
minute_appliances_status["DayOfYear"] = ((minute_appliances_status['Timestamp'] - minute_appliances_status['Timestamp'].min()) / (3600 * 12)).astype("int")

In [29]:
minute_appliances_status

,Timestamp,Load_StatusApplianceCooktop,Load_StatusApplianceDishwasher,Load_StatusApplianceOven,Load_StatusApplianceRangeHood,Load_StatusBA1Lights,Load_StatusBA2Lights,Load_StatusBR2Lights,Load_StatusBR3Lights,Load_StatusBR4Lights,...,Load_StatusPlugLoadMBRTV,Load_StatusPlugLoadSlowCooker,Load_StatusPlugLoadToaster,Load_StatusPlugLoadToasterOven,Load_StatusPlugLoadVacuum,Load_StatusPlugLoadVideoGame,Kitchen Lights,Living Room Lights,Master Bedroom Lights,DayOfYear
0,1372636874,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
1,1372636934,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
2,1372636994,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
3,1372637054,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
4,1372637114,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
518787,1404170962,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,729
518788,1404171022,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,729
518789,1404171082,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,729
518790,1404172762,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,729


In [30]:
appliance_job_list = []

for appliance in appliances_statusOnOff:
    name = " ".join(re.findall('[A-Z][^A-Z]*', appliance.split("Load_Status")[-1]))
    appliance_switch = minute_appliances_status[(minute_appliances_status[appliance] != minute_appliances_status[appliance].shift(1))][["Timestamp", "DayOfYear", appliance]][1:]

    df_indices = appliance_switch.index.tolist()
    for index, df_index in enumerate(df_indices):
        if appliance_switch.loc[df_index, appliance]:
            try:
                appliance_job_list.append({"start": appliance_switch.loc[df_index, "Timestamp"],
                                           "end": appliance_switch.loc[df_indices[index + 1], "Timestamp"],
                                           "appliance": name,
                                           "day": appliance_switch.loc[df_indices[index], "DayOfYear"]
                                           })
            except:
                continue

appliance_job_list

[{'start': 1372701631,
  'end': 1372702651,
  'appliance': 'Appliance Cooktop',
  'day': 1},
 {'start': 1372788028,
  'end': 1372789048,
  'appliance': 'Appliance Cooktop',
  'day': 3},
 {'start': 1372874427,
  'end': 1372875447,
  'appliance': 'Appliance Cooktop',
  'day': 5},
 {'start': 1372960826,
  'end': 1372961846,
  'appliance': 'Appliance Cooktop',
  'day': 7},
 {'start': 1373047228,
  'end': 1373048248,
  'appliance': 'Appliance Cooktop',
  'day': 9},
 {'start': 1373098407,
  'end': 1373099367,
  'appliance': 'Appliance Cooktop',
  'day': 10},
 {'start': 1373112027,
  'end': 1373113047,
  'appliance': 'Appliance Cooktop',
  'day': 10},
 {'start': 1373137106,
  'end': 1373138126,
  'appliance': 'Appliance Cooktop',
  'day': 11},
 {'start': 1373184807,
  'end': 1373185827,
  'appliance': 'Appliance Cooktop',
  'day': 12},
 {'start': 1373198427,
  'end': 1373199447,
  'appliance': 'Appliance Cooktop',
  'day': 12},
 {'start': 1373220026,
  'end': 1373221046,
  'appliance': 'Appli

In [31]:
df = pd.DataFrame(appliance_job_list).sort_values(by=['day', 'start']).reset_index(drop=True)
df

,start,end,appliance,day
0,1372649357,1372649537,M B A Lights,0
1,1372651097,1372651277,M B A Lights,0
2,1372658297,1372659197,M B A Lights,0
3,1372658297,1372666997,Kitchen Lights,0
4,1372658597,1372659017,Plug Load Blender,0
...,...,...,...,...
18641,1404165562,1404172762,Plug Load Fan,729
18642,1404165562,1404166462,Plug Load Heating Pad,729
18643,1404165562,1404169162,Master Bedroom Lights,729
18644,1404165862,1404167062,M B A Lights,729


In [32]:
df.to_csv(path_or_buf=Db.get_project_path("Project/TPM/Data/houseDataHalfday.csv"), header=False, index=False)